In [ ]:
import tensorflow as tf

from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

from sklearn.model_selection import train_test_split


In [ ]:
# Chargez le modèle InceptionV3 pré-entraîné
#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))


In [ ]:
# Prétraitement spécifique à InceptionV3
"""def preprocess_input_incep(x):
    x = preprocess_input(x)
    return x"""

In [ ]:
class CustomGenerator:
    def __init__(self, generator):
        self.generator = generator

    def generate_batches(self):
        while True:
            data = next(self.generator)
            yield data

In [121]:
# the path to dataset
dataset_train = 'D:\\workspace\\FER-PROJECT\\dataset\\train'
dataset_test = 'D:\\workspace\\FER-PROJECT\\dataset\\test'

#Set up a data generator with image augmentation for training
datagen = ImageDataGenerator(
    rescale=1./255,      # Normalize pixel values to be between 0 and 1
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

batch_size = 128

# Flow training images in batches using the datagen generator with target_size
train_generator = datagen.flow_from_directory(
    dataset_train,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    target_size=(75, 75)  # Resize images to 48 x 48
)
test_generator = datagen.flow_from_directory(
    dataset_test,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset=None,
    target_size=(75, 75)  # Resize images to 48 x 48

    )

# Save the relevant information to recreate the generator later
test_generator_info = {
    'directory': dataset_test,
    'batch_size': batch_size,
    'class_mode': 'categorical',
    'shuffle': False,
    'subset': None,
    'target_size': (75, 75)

}

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
print("Number of training batches per epoch:", len(train_generator))
print("Number of training samples:", len(train_generator.filenames))
print("Number of classes:", train_generator.num_classes)
print("Class labels:", train_generator.class_indices)

In [ ]:
# Create our personal model 
#model = Sequential()
# Ajouter le modèle pré-entraîné InceptionV3
#model.add(base_model)
# Ajouter une couche Flatten pour aplatir la sortie du modèle pré-entraîné
#model.add(Flatten())

In [ ]:
#Convolutional Layers 
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(48, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (5, 5), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())

In [ ]:
# Fully connected layers
model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['categorical_accuracy']
)

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
)

In [ ]:
model.save("model4.h5")

In [ ]:
# Recreate the pickled file with the highest protocol version(for later uses)
with open('test_generator_info.pkl', 'wb') as file:
    pickle.dump(test_generator_info, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Print the model summary
model.summary()

# Add print statements to check layer shapes
for layer in model.layers:
    print(layer.name, layer.output_shape)


In [ ]:

# Evaluate the model on the validation sets
validation_loss, validation_accuracy = model.evaluate(test_generator)
print("Validation Accuracy:", validation_accuracy)

# Get predictions on the validation set
y_pred = model.predict(test_generator)
y_true = test_generator.classes

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred.argmax(axis=1))

# Print classification report
print("Classification Report: \n" , classification_report(y_true, y_pred.argmax(axis=1)))

# Print confusion matrix
print("Confusion Matrix: \n ", conf_matrix)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)